In [1]:
!pip install -U sentence-transformers
!pip install transformers
!pip install trl
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.3 MB/s eta 0:00:00


In [2]:
from transformers import pipeline
from trl import SFTTrainer
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM
from transformers import pipeline, set_seed
from transformers import TrainingArguments

In [ ]:
# This is the baseline model we use to pass a query.
# We can use this as the control condition when we compare it to the results obtained by passing the same query
# and finetuning the LLM for Covid-19 Related FAQ
hf_id = 'gpt2' # @param{type:"string"}

base_generator = pipeline(
    task="text-generation",
    model= hf_id
)

query = 'Can Covid-19 be cured by increasing vitamin D in my diet?' # @param{type:"string"}
base_test = base_generator(query,
                 max_length=80 # @param{type:"integer"}
                           ,
                 early_stopping=True)
print(base_test)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:453: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "Can Covid-19 be cured by increasing vitamin D in my diet? Let's be sure if we get the vaccine (incomplete or incomplete with the vaccine), with a lot of vitamin D from the vitamin D supplement. Or, if we use the vitamin D supplement (very effective), without supplementation. A complete vitamin D diet can last many years. If you can get a vitamin D diet"}]


In [ ]:
dataset = load_dataset('json', data_files='/content/drive/MyDrive/Graduate/Courses/Winter 2024/EECS 6414/Datasets/qa_data.json', split="train")

model = AutoModelForCausalLM.from_pretrained(hf_id)
tokenizer = AutoTokenizer.from_pretrained(hf_id)
tokenizer.pad_token = tokenizer.eos_token

def formatting_prompts_func(example):
    output_texts = []
    for i in range(len(example['Question'])):
        text = f"""### Question: {example['Question'][i]}
        ### Answer: {example['Answer'][i]}"""
        output_texts.append(text)
    return output_texts

response_template = " ### Answer:"
collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)

training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/Graduate/Courses/Winter 2024/EECS 6414/model",
    auto_find_batch_size=True,
    num_train_epochs=5,
    learning_rate=1E-4,
    logging_steps=len(dataset['Question']),
    save_strategy='epoch'
    # fp16=True # Train Faster OwO
)

trainer = SFTTrainer(
    model,
    args=training_args,
    train_dataset=dataset,
    formatting_func=formatting_prompts_func,
    max_seq_length=512,
    data_collator=collator
)

trainer.train()

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/390 [00:00<?, ? examples/s]

Step,Training Loss


TrainOutput(global_step=147, training_loss=0.9703930705583015, metrics={'train_runtime': 1698.3613, 'train_samples_per_second': 0.689, 'train_steps_per_second': 0.087, 'total_flos': 45189232128000.0, 'train_loss': 0.9703930705583015, 'epoch': 3.0})

In [ ]:
# We will save this to test later
trainer.save_model('/content/drive/MyDrive/Graduate/Courses/Winter 2024/EECS 6414/model')
tokenizer.save_pretrained('/content/drive/MyDrive/Graduate/Courses/Winter 2024/EECS 6414/model')

('/content/drive/MyDrive/Graduate/Courses/Winter 2024/EECS 6414/model/tokenizer_config.json',
 '/content/drive/MyDrive/Graduate/Courses/Winter 2024/EECS 6414/model/special_tokens_map.json',
 '/content/drive/MyDrive/Graduate/Courses/Winter 2024/EECS 6414/model/vocab.json',
 '/content/drive/MyDrive/Graduate/Courses/Winter 2024/EECS 6414/model/merges.txt',
 '/content/drive/MyDrive/Graduate/Courses/Winter 2024/EECS 6414/model/added_tokens.json',
 '/content/drive/MyDrive/Graduate/Courses/Winter 2024/EECS 6414/model/tokenizer.json')

In [16]:
tokenizer = AutoTokenizer.from_pretrained('/content/drive/MyDrive/Graduate/Courses/Winter 2024/EECS 6414/model')
generator = pipeline('text-generation', model='/content/drive/MyDrive/Graduate/Courses/Winter 2024/EECS 6414/model', tokenizer=tokenizer)
query = "Can Covid-19 be caused by 5G Networks?" # @param {types:"string"}
response_length_in_chars = 125 # @param {type:"integer"}
number_of_responses = 10 # @param {type:"integer"}

generator(
    # Multinomial Sampling
    query,
    max_length=response_length_in_chars,
    num_return_sequences=number_of_responses,
    early_stopping=True,
    temperature=0.7,
    repetition_penalty=10.0,
    do_sample=True,
    num_beams=10,
    top_p=0.90,
    remove_invalid_values=True,
    truncation=True
    )

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Can Covid-19 be caused by 5G Networks? No, there is no scientific basis for the claim that 5G networks cause Covid-19. The virus primarily spreads through respiratory droplets when an infected person coughs, sneezes, or talks. There is no evidence to support the theory that 5G networks play a role in its transmission. Vaccination plays a crucial role in maintaining immunity against viruses such as SARS-CoV-2 and Guillain-Barré Syndrome (GCV-2). Vaccination also helps reduce the risk of contracting the virus while providing protection against severe illness'},
 {'generated_text': 'Can Covid-19 be caused by 5G Networks? No, there is no scientific basis for the claim that 5G networks cause Covid-19. The virus primarily spreads through respiratory droplets when an infected person coughs, sneezes, or talks. There is no evidence to support the theory that 5G networks play a role in its transmission. Vaccination plays a crucial role in maintaining immunity against viruses